# DOWNLOAD DATASET

In [1]:
import requests, zipfile
# The url and auth_value from the website
myobj = {'auth_token': ''}
base_path='https://api.zindi.africa/v1/competitions/fossil-stock-forecasting-challenge/files/'
def load_zindi_data(data_list,base_path,target_dir):
  for data in data_list:
      target_path=  target_dir +data
      data_path=base_path+ data
      x = requests.post(data_path, data = myobj,stream=True)
      handle = open(target_path, "wb")
      for chunk in x.iter_content(chunk_size=512):
        if chunk:  # filter out keep-alive new chunks
          handle.write(chunk)
      handle.close()

data_list = ['Train.csv','Test.csv','DataDictionary.csv', 'SampleSubmission.csv']
load_zindi_data(data_list,base_path,'/content/')

# Load Data 


In [2]:
import numpy as np,pandas as pd,matplotlib.pyplot as plt,seaborn as sns
from sklearn import ensemble

train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
data_dict = pd.read_csv('DataDictionary.csv')
sample_sub = pd.read_csv('SampleSubmission.csv')


In [3]:
train.head(10)

,sku_name,starting_inventory,sellin,sellin_channel_1,sellin_channel_2,sellin_channel_3,sellin_channel_4,sellin_channel_5,sellin_channel_6,sellin_channel_7,...,month,year,product_lifecycle_stage,FLAG100,disc_month,cum_disc,CAT_GENDER_BOTH,CAT_GENDER_MEN,CAT_GENDER_WOMEN,Weeks
0,YOSHWARDTERR,0,1013,0,0,0,1013,0,0,0,...,7,2016,U,0.000000,0,0,0,1,0,1
1,YOSHWARDTERR,0,2026,0,0,0,2026,0,0,0,...,2,2017,U,0.000000,0,0,0,1,0,0
2,YOSHWARDTERR,0,1013,0,0,0,1013,0,0,0,...,5,2017,U,0.000000,0,0,0,1,0,0
3,YOSHUANEMARX,0,320108,4052,40520,240081,4052,6078,18234,0,...,4,2018,W,0.270966,1,1,0,0,1,1
4,YOSHUANEMARX,0,132703,2026,0,81040,3039,25325,18234,0,...,5,2018,W,0.063004,0,1,0,0,1,0
5,YOSHUANEMARX,0,168158,0,0,151950,6078,0,8104,0,...,6,2018,W,0.000000,0,1,0,0,1,0
6,YOSHUANEMARX,0,43559,0,6078,34442,0,0,0,0,...,7,2018,W,0.000000,0,0,0,0,1,1
7,YOSHUANEMARX,0,22286,0,0,17221,4052,0,0,0,...,8,2018,W,0.000000,0,0,0,0,1,0
8,YOSHUANEMARX,0,146885,0,9117,137768,0,0,0,0,...,9,2018,W,0.000000,0,0,0,0,1,1
9,YOSHUANEMARX,0,46598,1013,13169,31403,0,0,0,0,...,10,2018,W,0.000000,0,0,0,0,1,0


In [4]:
train.shape,test.shape

((44907, 45), (1528, 6))

In [5]:
data_dict

,Column Name,Description
0,sku_name,Product SKU
1,starting_inventory,Starting inventory at the beginning of the mon...
2,sellin,The sellin or demand. What is sold in to our c...
3,sellin_channel_1,The sellin for channel 1
4,sellin_channel_2,The sellin for channel 2
5,sellin_channel_3,The sellin for channel 3
6,sellin_channel_4,The sellin for channel 4
7,sellin_channel_5,The sellin for channel 5
8,sellin_channel_6,The sellin for channel 6
9,sellin_channel_7,The sellin for channel 7
